In [40]:
using Pkg
Pkg.activate(".")
using Revise, PtyLab, TestImages, ImageShow, IndexFunArrays, FFTW, Napari, HDF5, Noise, FourierTools

  Activating environment at `~/Documents/Uni/my_projects/Ptychography/PtyLab.jl/examples/Project.toml`


## Object

In [41]:
img_abs = Float32.(testimage("fabio_gray_512"))[200:380, 200:380]
img_phase = Float32.(testimage("resolution_test_512"))[200-30:380-30, 200+30:380+30]
object = img_abs .* cispi.(2 .* img_phase)

complex_show(object);

## Random Grid

In [38]:
grid_size = size(object)
tile_size = (70, 70)

grr = PtyLab.grid_regular_rand(grid_size, tile_size, (18, 18), 30);
@show grr.overlap
show_grid(grr, only_points=false);

grr.overlap = (0.9142857142857143, 0.9142857142857143)


## Probe

In [50]:
probe = IndexFunArrays.gaussian(Float32, tile_size, scale=0.010) .* cis.(Float32(2π) .* 
     4 .* gaussian(Float32, tile_size, scale=0.003));

complex_show(probe);

## Simulate and store as dataset

In [39]:
ptychogram = zeros(Float32, (tile_size..., length(grr.tiles)));
p = Params()
o2d, d2o = Fraunhofer(probe, fftshiftFlag=true);

for (i, t) in enumerate(grr.tiles)
    ptychogram[:, :, i] = abs2.(o2d(view(object, t.i₁:t.i₂,  t.j₁:t.j₂) .* probe))#, 200000)
end


lambda = 633f-9
z = 50f-3
dxd = 10f-6
Nd = size(ptychogram, 1)
dxo = lambda * z / (Nd * dxd)

fid_new = h5open("simulated_ptychography.hdf5", "w");
fid_new["Nd"] = Nd
fid_new["No"] = size(img_abs, 1)
fid_new["dxd"] = 10f-6
fid_new["encoder"] = PtyLab.encoder(grr, dxo, offset=(50, 50))
fid_new["wavelength"] = lambda
fid_new["entrancePupilDiameter"] = dxo * 30
fid_new["zo"] = z
fid_new["ptychogram"] = ptychogram
close(fid_new)
#@view_image ptychogram;

## Load dataset again

In [42]:
experimentalData = ExperimentalDataCPM("simulated_ptychography.hdf5");

reconstruction = ReconstructionCPM(experimentalData);
reconstruction = PtyLab.initializeObjectProbe!(reconstruction);

engine = PtyLab.ePIE()
params2 = Params()

┌ Info: Reading HDF5.File: (read-only) simulated_ptychography.hdf5 was successful
└ @ PtyLab /home/fxw/Documents/Uni/my_projects/Ptychography/PtyLab.jl/src/ExperimentalData.jl:55
┌ Warning: Currently No = 4 * Np, just as guess, should be improved
└ @ PtyLab /home/fxw/Documents/Uni/my_projects/Ptychography/PtyLab.jl/src/utils_calc.jl:91


Params
  fftshiftFlag: Bool false
  transposePtychogram: Bool true
  intensityConstraint: PtyLab.IntensityConstraintStandard <: PtyLab.IntensityConstraint
  propagatorType: Fraunhofer (function of type typeof(Fraunhofer))
  randPositionOrder: Bool true
  comStabilizationSwitch: Bool true


In [49]:
#engines.
params2.fftshiftFlag = false
engine.betaProbe = 0.75f0
engine.betaObject = 0.75f0

params2.transposePtychogram = false
params2.comStabilizationSwitch = true
reconstruction = PtyLab.initializeObjectProbe!(reconstruction);
engine.numIterations = 50

@time p, o = PtyLab.reconstruct(engine, params2, reconstruction);

┌ Warning: gimmel is currently estimated as `100 * eps(Float32)`
└ @ PtyLab /home/fxw/Documents/Uni/my_projects/Ptychography/PtyLab.jl/src/Engines.jl:54
Progress: 100%|█████████████████████████████████████████| Time: 0:00:04


  4.542010 seconds (2.10 M allocations: 7.809 GiB, 5.00% gc time)


In [48]:
complex_show(o[:, :, 1,1,1,1]);

In [47]:
complex_show(p[:, :, 1,1,1,1]);